In [1]:
# organize the imports
import opentrons.execute
from opentrons import types 
import time
import numpy as np


In [2]:
#!pip3 install zeroconf
#!pip3 install requests

# Connect to the microscope 
from microscope_client import MicroscopeClient 
microscope = MicroscopeClient(host="192.168.178.33", port=5000)


In [3]:
import opentrons
opentrons.__file__

'/usr/lib/python3.7/site-packages/opentrons/__init__.pyc'

In [4]:
# Connect to the opentrons robot
protocol = opentrons.execute.get_protocol_api('2.9')
protocol.home()

# setup the pipets
pipette = protocol.load_instrument('p300_single_gen2', 'right')
pipette_8 = protocol.load_instrument('p300_multi', 'left') # causes errors -> don't use two pipettes!!


In [5]:
# setup the labware in the trays
#plate_microscope = protocol.load_labware('corning_96_wellplate_360ul_flat', location='5')
#plate_reagents = protocol.load_labware('corning_24_wellplate_3.4ml_flat', location='9')
plate_reagents = protocol.load_labware('opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap', location='10')


# Move light to the sample 
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul',location='11')

# sample plate on microscope
# offset of the well plate in mm
plate_sample = protocol.load_labware('corning_96_wellplate_360ul_flat', location='3')

In [51]:
from opentrons.types import Location, Point

# set speed
#protocol.max_speeds['X'] = 200 # default = 400
#protocol.max_speeds['Y'] = 200 # default = 300


# ######
# 0 4 8
# 1 5 9
# 2 6 10 
# 3 7 11

# wells with reagents
i_phal = 0
i_sir = 1
i_pbs_1 = 2
i_trash = 5

# general aspiration volume
V_aspirate = 200

# move it down since opentrons expects the reagents to be furthter away from the ground
offset_z_reagents = -65

# define locations 
minimum_z_height = 150 # minimum height for arcs driven by the robot
position_sample_light = (252, 100, 210)

# compute the positions
position_idle =  (200,200,140)#
position_phal = list(plate_reagents.well(i_phal).center()._point)
position_sir = list(plate_reagents.well(i_sir).center()._point)


#position_trash = list(plate_reagents.well(i_trash).center()._point)
#position_trash = list(pipette._trash._implementation._calibrated_offset)

def move_to_coord(pipette, position_xyz, offset = (0,0,0), minimum_z_height=190):
    if pipette._last_tip_picked_up_from is None:
        print("Please add a pipette first!")
        
        
    position_final = (position_xyz[0]+offset[0],
                        position_xyz[1]+offset[1],
                        position_xyz[2]+offset[2])
    print("Moving to: "+str(position_final))                       
    pipette.move_to(Location(Point(*position_final), None),minimum_z_height=minimum_z_height)

    
def pickup_fresh_pipette_tip(i_pipette: int, pipette, tiprack, offset_pipette_rack):
    
    # check if pipette is attached
    try:
        if pipette._last_tip_picked_up_from is not None:
            pipette.drop_tip()
    except:
        print("Something went wrong")
    my_fresh_tip = tiprack.wells()[i_pipette]
    adjusted_location = my_fresh_tip.center().move(types.Point(x=offset_pipette_rack[0], y=offset_pipette_rack[1], z=offset_pipette_rack[2]))
    #_pipette.pick_up_tip(_adjusted_location)
    #pipette.move_to(_adjusted_location)
    try:
        pipette.pick_up_tip(adjusted_location)
    except:
        pipette.drop_tip()
        pipette.pick_up_tip(adjusted_location)
    
    print("My pipette number: "+str(i_pipette)+"and type: "+str(type(i_pipette)))
    return i_pipette + 1
    



def takesnapshot(pipette, position_sample_light, offset=(2,-2,0), I_led=1, I_laser=0):
    microscope.set_laser_led(I_laser,I_led)
    move_to_coord(pipette, position_sample_light, offset=offset, minimum_z_height=minimum_z_height)
    microscope.capture_image_to_disk()
    microscope.set_laser_led(0,0)
    
def wellscan(microscope, offset_x, offset_y, i_experiment,
        Nx=3, Ny=3, well_to_well_steps = 9000,
        autofocus_dz=2000, autofocus_Nz=11,
            name_experiment="wellscan_",
            focus_pos_list=None, 
            is_autofocus=True, is_autofocus_fine = False,
            is_autofocus_fast = False,
            I_laser=0, I_led=1):
    """
    Save a set of images in a wellscan

    Args:
        microscope: Microscope object
        offset_x (int): Number of images to take
        offset_y (int/float): Time, in seconds, between sequential captures
    """
    folder = "wellscan_"

    # cast variables
    Nx = int(Nx)
    Ny = int(Ny)
    well_to_well_steps = int(well_to_well_steps)
    
    # get current z-position for the stage
    offset_z = microscope.position["z"]

    # move to the home positoin
    print("Start moving to the position")
    
    # go to the first location
    microscope.move((offset_x,offset_y,offset_z), absolute=True)

    # in case we have focus positions available, skip it
    print("Start scan")
    #%%
    if focus_pos_list is None:
        focus_pos_list = []
        
    folder = "WELLSCAN_"

    # preset variables
    i_image = 0
    i_well = 0
    last_offset_z_row = offset_z
    for wellpos_y in range(Ny):
        for wellpos_x in range(Nx):

            if last_offset_z_row == 0:
                offset_z = last_offset_z_row

            print("Move microscope")
            current_x, current_y = offset_x+well_to_well_steps*wellpos_x,offset_y+well_to_well_steps*wellpos_y

            if is_autofocus:
                microscope.move((current_x, current_y, offset_z), absolute=True)
                
                # turn on laser/led and perform autofocus
                microscope.set_laser_led(I_laser,I_laser)
                print("ERROR make sure the correct laser itnensity is assigned!")
                if is_autofocus_fast:
                    offset_z = autofocus_fast(autofocus_dz)
                else:
                    offset_z = autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz, is_uselight=True)
                
                
                if is_autofocus_fine:
                    microscope.move((current_x, current_y, offset_z), absolute=True)

                    # turn on laser/led and perform autofocus
                    offset_z = autofocus_coarse(dz=200, nz=5, is_uselight=True)
                microscope.set_laser_led(0,0)                
                focus_pos_list.append(offset_z)

                if last_offset_z_row == 0:
                    last_offset_z_row = offset_z
            else:
                try:
                    offset_z = focus_pos_list[i_well]
                except:
                    offset_z = microscope.position["z"]
                microscope.move((current_x, current_y, offset_z), absolute=True)

            print("offset_z:"+str(offset_z))
            
            # debounce
            time.sleep(.5)
            
            # turn on light and take an image
            if I_laser > 0:
                params = {
                    "use_video_port": True,
                    "bayer": False,
                    "temporary" :False,
                    "filename": name_experiment+"FLOU_"+str(i_experiment)+"_"+
                                str(i_well)+"_"+str(i_image)+
                                "_"+str(wellpos_x)+"_"+str(wellpos_y), 
                }
                microscope.set_laser_led(0,1)
                microscope.capture_image_to_disk(params)
            if I_led > 0:             
                params = {
                    "use_video_port": True,
                    "bayer": False,
                    "temporary" :False,
                    "filename": name_experiment+"BF_"+str(i_experiment)+"_"+
                                str(i_well)+"_"+str(i_image)+
                                "_"+str(wellpos_x)+"_"+str(wellpos_y), 
                }
                microscope.set_laser_led(0,0)
                microscope.capture_image_to_disk(params)
            microscope.set_laser_led(0,0)            
            i_image += 1
            i_well += 1

    i_experiment += 1
    return focus_pos_list

def autofocus_coarse(dz=1000, nz=11, is_uselight=False):
    if False : #is_uselight:
        microscope.set_laser_led(0,1)
        time.sleep(.2)
    payload = {'dz': list(np.linspace(-dz,dz,nz))}
    print("Focusing: "+str(payload))
    microscope.extensions["org.openflexure.autofocus"]["autofocus"].post_json(payload)
    if False:# is_uselight:
        microscope.set_laser_led(0,1)
    return microscope.position['z']

def autofocus_fast(dz):
    params={'dz': dz}
    microscope.extensions["org.openflexure.autofocus"]["fast_autofocus"].post_json(params)
    return microscope.position['z']


In [16]:
position_trash =  (365,350, 100)#
move_to_coord(pipette, position_trash, offset=(0,0,0), minimum_z_height=minimum_z_height)

Moving to: (365, 350, 100)


In [86]:
position_idle =  (100,100, 150)#
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)



Moving to: (100, 100, 150)


In [85]:
microscope.set_laser_led(1,1)


Please add a pipette first!
Moving to: (250, 97, 202)


In [12]:
protocol.home()
i_pipette  = 0

# Calibrate

In [68]:
#protocol.home()
# calibrate tiprack

#pitty, but moving the xy coordinates doesn't work
print(i_pipette)
offset_pipette_rack=(1,7,0)
tiprack.set_calibration(Point(*offset_pipette_rack))

i_pipette=pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack=offset_pipette_rack)
print(type(i_pipette))

7
My pipette number: 7and type: <class 'int'>
<class 'int'>


In [82]:
# calibrate illumination poisiton

position_sample_light = (250, 97, 202)
offset_x = 3000 # 1290+0*9000
offset_y = 27000 # -1400+4*9000
offset_z = microscope.position["z"]
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
microscope.set_laser_led(0,0)
microscope.move((offset_x,offset_y,offset_z),absolute=True)
autofocus_dz = 500
autofocus_Nz = 7
#offset_z = autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)
microscope.set_laser_led(0,0)

Please add a pipette first!
Moving to: (250, 97, 202)


In [57]:
# pipette should dip in the wells on the microscope
#pipette.home()

i_pipette=pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack=offset_pipette_rack)
offset_z = microscope.position["z"]
home_position_microscope=(0,0,offset_z)
microscope.move((home_position_microscope))

offset_well_x = -29
offset_well_y = 68
offset_well_z = 124

row_phal_sample = 3
row_sir_sample = 4
for i_pos in range(4):
    position_sample_i = np.array(plate_sample.rows()[i_pos][row_phal_sample].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)

for i_pos in range(4):
    position_sample_i = np.array(plate_sample.rows()[i_pos][row_sir_sample].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)

# move out of the well again
move_to_coord(pipette, position_sample_i, offset=(0,0,20), minimum_z_height=minimum_z_height)

    

My pipette number: 5and type: <class 'int'>
Moving to: (277.38, 142.24, 132.885)
Moving to: (277.38, 133.24, 132.885)
Moving to: (277.38, 124.24000000000001, 132.885)
Moving to: (277.38, 115.24000000000001, 132.885)
Moving to: (286.38, 142.24, 132.885)
Moving to: (286.38, 133.24, 132.885)
Moving to: (286.38, 124.24000000000001, 132.885)
Moving to: (286.38, 115.24000000000001, 132.885)
Moving to: (286.38, 115.24000000000001, 152.885)


In [58]:
# calibrate reagetn slots
#pipette.home()
offset_x_reagents = -4
offset_y_reagents = 10
offset_z_reagents = -55 # smaller number means: go up
for i_pos in range(4):
    #i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, -8)
    position_reagent_i = np.array(plate_reagents.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)

# move out of the well again
move_to_coord(pipette, position_reagent_i, offset=(0,0,20), minimum_z_height=minimum_z_height)

    

Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (14.21, 337.65, 5.949999999999996)
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (14.21, 299.09, 5.949999999999996)
Moving to: (18.21, 289.09, 80.94999999999999)


Please add a pipette first!
Moving to: (248, 150, 192)


In [38]:
# take BF image

move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
time.sleep(1) # debounce
microscope.move((offset_x,offset_y,offset_z),absolute=True)
microscope.capture_image_to_disk()

# take Fluo image
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
time.sleep(1) # debounce
microscope.set_laser_led(0,1)
microscope.capture_image_to_disk()
time.sleep(1)
microscope.set_laser_led(0,0)

Please add a pipette first!
Moving to: (250, 97, 202)
Moving to: (100, 100, 150)


In [39]:
microscope.set_laser_led(0,1)
autofocus_dz = 200
autofocus_Nz = 7
offset_z = autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)
microscope.set_laser_led(0,0)

Focusing: {'dz': [-200.0, -133.33333333333331, -66.66666666666666, 0.0, 66.66666666666669, 133.33333333333337, 200.0]}


In [ ]:
microscope.set_laser_led(0,0)

i_experiment = 0


from datetime import datetime
timestamp = datetime.now().strftime("%Y_%m_%d-%I-%M-%S_%p")


# Do a time-lapse for an hour

In [47]:
# Microscope coordinates

Nx=8
Ny=3
well_to_well_steps = 9000
autofocus_dz = 750
autofocus_Nz= 7

mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list

while (True): #(time.time()-mytime)<(t_duration*60)):

    if (time.time()-mytime)>(t_duration*60):
        break
    
    if (i_experiment % 20)==0:
        is_autofocus=True
    else:
        is_autofocus=False
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_"+timestamp,
                focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                is_autofocus_fine = False, 
                is_autofocus_fast = False,
                I_laser=0, I_led=1)
    i_experiment += 1

Please add a pipette first!
Moving to: (250, 97, 202)
Start moving to the position
Start scan
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-750.0, -500.0, -250.0, 0.0, 250.0, 500.0, 750.0]}
offset_z:776
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-750.0, -500.0, -250.0, 0.0, 250.0, 500.0, 750.0]}
offset_z:386
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-750.0, -500.0, -250.0, 0.0, 250.0, 500.0, 750.0]}
offset_z:218
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-750.0, -500.0, -250.0, 0.0, 250.0, 500.0, 750.0]}
offset_z:66
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-750.0, -500.0, -250.0, 0.0, 250.0, 500.0, 750.0]}
offset_z:-60
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-750.0, -500.0, -250.0, 0.0, 250.0, 500.0

In [48]:
microscope.set_laser_led(0,0)

# Add Alexa Fluor 647 Phalloidin

In [59]:

# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
#i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

print(type(i_pipette))
V_aspirates_phal = (1,2,5,10,25,50,75,100)
for i_pos in range(len(plate_sample.rows())):
    
    # 2. Pickup Reagent depending on current iteration
    position_reagent_i = np.array(plate_reagents.wells()[i_phal].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    time.sleep(1)
    pipette.aspirate(V_aspirates_phal[i_pos])
    print("Aspirating: "+str(V_aspirates_phal[i_pos]))
    
    # 3. Move to Sample location and mix it with the sample
    position_sample_i = np.array(plate_sample.rows()[i_pos][row_phal_sample].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()

# 4. Move the pipette out 
position_sample_i = np.array(plate_sample.rows()[i_pos][i_phal].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
move_to_coord(pipette, position_sample_i, offset=(0,0,20), minimum_z_height=minimum_z_height)



<class 'int'>
Moving to: (14.21, 356.93, 5.949999999999996)
Aspirating: 1
Moving to: (277.38, 142.24, 132.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Aspirating: 2
Moving to: (277.38, 133.24, 132.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Aspirating: 5
Moving to: (277.38, 124.24000000000001, 132.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Aspirating: 10
Moving to: (277.38, 115.24000000000001, 132.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Aspirating: 25
Moving to: (277.38, 106.24000000000001, 132.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Aspirating: 50
Moving to: (277.38, 97.24, 132.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Aspirating: 75
Moving to: (277.38, 88.24, 132.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Aspirating: 100
Moving to: (277.38, 79.24, 132.885)
Moving to: (250.38, 79.24, 152.885)


# Add SiR

In [60]:
V_aspirate = 100

# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

print(type(i_pipette))
V_aspirates_phal = (1,5,10,20,40,60,80,200)
for i_pos in range(len(plate_sample.rows())):
    
    # 2. Pickup Reagent depending on current iteration
    position_reagent_i = np.array(plate_reagents.wells()[i_sir].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    time.sleep(1)
    pipette.aspirate(V_aspirates_phal[i_pos])
    print("Aspirating: "+str(V_aspirates_phal[i_pos]))
    
    # 3. Move to Sample location and mix it with the sample
    position_sample_i = np.array(plate_sample.rows()[i_pos][row_sir_sample].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()

# 4. Move the pipette out 
position_sample_i = np.array(plate_sample.rows()[i_pos][i_sir].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
move_to_coord(pipette, position_sample_i, offset=(0,0,20), minimum_z_height=minimum_z_height)



My pipette number: 6and type: <class 'int'>
<class 'int'>
Moving to: (14.21, 337.65, 5.949999999999996)
Aspirating: 1
Moving to: (286.38, 142.24, 132.885)
Moving to: (14.21, 337.65, 5.949999999999996)
Aspirating: 5
Moving to: (286.38, 133.24, 132.885)
Moving to: (14.21, 337.65, 5.949999999999996)
Aspirating: 10
Moving to: (286.38, 124.24000000000001, 132.885)
Moving to: (14.21, 337.65, 5.949999999999996)
Aspirating: 20
Moving to: (286.38, 115.24000000000001, 132.885)
Moving to: (14.21, 337.65, 5.949999999999996)
Aspirating: 40
Moving to: (286.38, 106.24000000000001, 132.885)
Moving to: (14.21, 337.65, 5.949999999999996)
Aspirating: 60
Moving to: (286.38, 97.24, 132.885)
Moving to: (14.21, 337.65, 5.949999999999996)
Aspirating: 80
Moving to: (286.38, 88.24, 132.885)
Moving to: (14.21, 337.65, 5.949999999999996)
Aspirating: 200
Moving to: (286.38, 79.24, 132.885)
Moving to: (259.38, 79.24, 152.885)


# Do timelapse for an hour

In [71]:
# Microscope coordinates
mytime = time.time()
t_duration = 60 # minutes
autofocus_dz = 1000
autofocus_Nz= 9
while (True): #(time.time()-mytime)<(t_duration*60)):

    if (time.time()-mytime)>(t_duration*60):
        break
    
    if focus_pos_list is None or (i_experiment % 60)==0:
        is_autofocus=True
    else:
        is_autofocus=False
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_"+timestamp,
                focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                is_autofocus_fine = False, 
                is_autofocus_fast = False,
                I_laser=0, I_led=1)
    i_experiment += 1

Please add a pipette first!
Moving to: (250, 97, 202)
Start moving to the position
Start scan
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-1000.0, -750.0, -500.0, -250.0, 0.0, 250.0, 500.0, 750.0, 1000.0]}
offset_z:-343
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-1000.0, -750.0, -500.0, -250.0, 0.0, 250.0, 500.0, 750.0, 1000.0]}
offset_z:-596
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-1000.0, -750.0, -500.0, -250.0, 0.0, 250.0, 500.0, 750.0, 1000.0]}
offset_z:-763
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-1000.0, -750.0, -500.0, -250.0, 0.0, 250.0, 500.0, 750.0, 1000.0]}
offset_z:-900
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-1000.0, -750.0, -500.0, -250.0, 0.0, 250.0, 500.0, 750.0, 1000.0]}
offset_z:-1050
Move microscope
ERROR make sure the corr

# Wash sample 3x 

In [87]:
i_pipette=pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack=offset_pipette_rack)

# add primary to sample
V_aspirate = 180

# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

V_eppi = 2000
V_aspirated = 0
i_wash = 0
for _ in range(3):
    # Do that 3 times
    for i_pos in range(len(plate_sample.rows()[0:2])*8):
        # 2. Move to Sample location and aspirate liquid 
        position_sample_i = np.array(plate_sample.wells()[i_pos+24].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z-1))
        move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
        pipette.aspirate(V_aspirate)
        #
        # 3. Trash the liquid
        move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
        pipette.dispense()
        
        # 4. grab PBS and aspirate
        if (V_aspirated+V_aspirate)>V_eppi:
            i_wash += 1
            V_aspirated=0
        print("Reagent: ")
        position_reagent_i = np.array(plate_reagents.wells()[i_pbs_1+i_wash].center()._point)
        move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
        time.sleep(.2)
        pipette.aspirate(V_aspirate)
        V_aspirated+=V_aspirate

        # 5. go to microscope and dispense
        position_sample_i = np.array(plate_sample.wells()[i_pos].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
        move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
        pipette.dispense()

My pipette number: 10and type: <class 'int'>
Moving to: (277.38, 142.24, 131.885)
Moving to: (361, 360, 55)
Reagent: 
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (250.38, 142.24, 132.885)
Moving to: (277.38, 133.24, 131.885)
Moving to: (361, 360, 55)
Reagent: 
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (250.38, 133.24, 132.885)
Moving to: (277.38, 124.24000000000001, 131.885)
Moving to: (361, 360, 55)
Reagent: 
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (250.38, 124.24000000000001, 132.885)
Moving to: (277.38, 115.24000000000001, 131.885)
Moving to: (361, 360, 55)
Reagent: 
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (250.38, 115.24000000000001, 132.885)
Moving to: (277.38, 106.24000000000001, 131.885)
Moving to: (361, 360, 55)
Reagent: 
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (250.38, 106.24000000000001, 132.885)
Moving to: (277.38, 97.24, 131.885)
Moving to: (361, 360, 55)
Reagent: 
Moving to: (14.21, 318.37, 5.94

# Do a timelapse with fluorescence

In [76]:
microscope.set_laser_led(0,0)

In [93]:
# Microscope coordinates

mytime = time.time()
t_duration = 20 # minutes
#focus_pos_list = None # start with a fresh list

i_fluo = 0
while (True): #(time.time()-mytime)<(t_duration*60)):

    if (time.time()-mytime)>(t_duration*60):
        break
    
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    if i_fluo==0:
        is_autofocus = True
        I_led = 0
        I_laser = 0
        move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    else:
        is_autofocus=False
        I_laser = 1
        I_led = 1
        move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
    i_fluo+=1
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_"+timestamp,
                focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                is_autofocus_fine = False, 
                is_autofocus_fast = False,
                I_laser=I_laser, I_led=I_led)
    i_experiment += 1

Moving to: (100, 100, 150)
Start moving to the position
Start scan
Move microscope
offset_z:-117
Move microscope
offset_z:-598
Move microscope
offset_z:-753
Move microscope
offset_z:-924
Move microscope
offset_z:-697
Move microscope
offset_z:-1085
Move microscope
offset_z:-1228
Move microscope
offset_z:-1386
Move microscope
offset_z:-472
Move microscope
offset_z:-891
Move microscope
offset_z:-713
Move microscope
offset_z:-1098
Move microscope
offset_z:-915
Move microscope
offset_z:-1287
Move microscope
offset_z:-1479
Move microscope
offset_z:-1468
Move microscope
offset_z:-827
Move microscope
offset_z:-987
Move microscope
offset_z:-1140
Move microscope
offset_z:-1211
Move microscope
offset_z:-1351
Move microscope
offset_z:-1518
Move microscope
offset_z:-1510
Move microscope
offset_z:-1655
Moving to: (100, 100, 150)
Start moving to the position
Start scan
Move microscope
offset_z:-117
Move microscope
offset_z:-598
Move microscope
offset_z:-753
Move microscope
offset_z:-924
Move microsco

In [ ]:
# Microscope coordinates

mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list

while (True): #(time.time()-mytime)<(t_duration*60)):

    
    if focus_pos_list is None or (i_experiment % 30)==0:
        is_autofocus=True
    else:
        is_autofocus=False
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_"+timestamp,
                focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                is_autofocus_fine = False, 
                is_autofocus_fast = False,
                I_laser=0, I_led=1)
    i_experiment += 1

Please add a pipette first!
Moving to: (250, 97, 202)
Start moving to the position
Start scan
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-500.0, -333.33333333333337, -166.66666666666669, 0.0, 166.66666666666663, 333.33333333333326, 500.0]}
offset_z:544
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-500.0, -333.33333333333337, -166.66666666666669, 0.0, 166.66666666666663, 333.33333333333326, 500.0]}
offset_z:1044
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-500.0, -333.33333333333337, -166.66666666666669, 0.0, 166.66666666666663, 333.33333333333326, 500.0]}
offset_z:1203
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-500.0, -333.33333333333337, -166.66666666666669, 0.0, 166.66666666666663, 333.33333333333326, 500.0]}
offset_z:-144
Move microscope
ERROR make sure the correct laser itnensity is assigned!
Fo